# PV Degradation Tools Tutorial - 2
### Degradation and Solder Fatigue
***
Matthew Brown

2022.12.29
***

**Requirements:**
- spectral irradiance (measured or simulated)
- wavelengths of spectral irradiance data
- module RH
- module temperature


**Objectives:**
1. Read in spectral irradiance
2. Calculate relative backsheet degradation
3. Calculate Solder Fatigue

**Miscillaneous:**

In [1]:
import pvlib
import pandas as pd
import numpy as np
import PVDegradationTools as PVD

### 1. Read in spectral irradiance data

In [7]:
input_file = r'..\..\tests\test_weatherandspectra.csv'

input_df = pd.read_csv(input_file, header=0, index_col=1)
input_df.head()

,Unnamed: 0,Spectra,Temperature,RH
timestamp,,,,
2021-03-09 10:00:00,0,"[0.6892146677599185, 0.40215646494410884, 0.67...",45,59
2021-03-09 11:00:00,1,"[0.15575709102178648, 0.5464374649246564, 0.68...",44,56
2021-03-09 12:00:00,2,"[0.22782105874481207, 0.9056495270031296, 0.26...",59,39
2021-03-09 13:00:00,3,"[0.3741943134512433, 0.035830980984344674, 0.4...",44,13
2021-03-09 14:00:00,4,"[0.40321187996337626, 0.6473167864022122, 0.69...",25,39


### 2. Calculate Degradation

Normally, we'd have to find the module temperature (with PVLIB) and the module RH (with PVD) but we'll assume the dataset above is providing that data.

In [9]:
wavelengths = np.array(range(280,420,20))
degradation = PVD.Degradation.degradation(spectra=input_df['Spectra'], rh_module=input_df['RH'],
                                            temp_module=input_df['Temperature'],
                                            wavelengths=wavelengths)

USING THE EXCEPT in PVDegradations Degradation function


### 5. Solder Fatigue

Solder Fatigue will evaluat the damage (in kPa) over a time series. Each input must be a series
with a common index or arrays of equal length.

There are several optional parameters for the empirical formula. For more information on these 
parameters, refer to the original paper: https://www.sciencedirect.com/science/article/pii/S0026271416300609#t0020.

The default values come from figure 4 of the above paper
- Scaling Constant, C1 = 405.6
- Activation Energy, Q = 0.12 eV
- fit parameter, n = 1.9
- fit parameter, b = 0.33

In [ ]:
sam_file = r'psm3_demo.csv'
data, meta = pvlib.iotools.read_psm3(filename=sam_file)

time_range = data.index
dni = data.DNI
dhi = data.DHI
ghi = data.GHI

solar_pos = pvlib.solarposition.get_solarposition(time=time_range, latitude=meta['Latitude'],
                                            longitude=meta['Longitude'], altitude=meta['Elevation'])

poa_df = pvlib.irradiance.get_total_irradiance(surface_tilt=20, surface_azimuth=270, 
                                                    solar_zenith=solar_pos.apparent_zenith,
                                                    solar_azimuth=solar_pos.azimuth,
                                                    dni=dni, ghi=ghi, dhi=dhi)
poa_global = poa_df.poa_global

cell_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']
temp_cell = pvlib.temperature.sapm_cell(poa_global=0, temp_air=data['Temperature'],
                                        wind_speed=data['Wind Speed'], **cell_params)

damage = PVD.Degradation.solder_fatigue(time_range=time_range, temp_cell=temp_cell)

c:\users\mbrown2\documents\github\pvlib\pvlib\iotools\psm3.py:349: pvlibDeprecationWarning: PSM3 variable names will be renamed to pvlib conventions by default starting in pvlib 0.11.0. Specify map_variables=True to enable that behavior now, or specify map_variables=False to hide this warning.
  warnings.warn(
